**Task 04: Graph querying**

In [1]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2021-2022/master/Assignment4/course_materials"


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from rdflib import Graph, Namespace, Literal
g = Graph()
g.parse("ejemplito.ttl", format="ttl")

<Graph identifier=Ne9216304e90041d68b8264f9c23b22fc (<class 'rdflib.graph.Graph'>)>

Listamos todos los recursos que contienen la propiedad VCARD:FN

In [3]:
from rdflib.plugins.sparql import prepareQuery

mv= Namespace("http://schema.mobivoc.org/#")
ns= Namespace("http://HO-G1.linkeddata.es/ontology/Chargingstation#")
uiote= Namespace("http://www.w3id.org/urban-iot/uiote#")
schema= Namespace("http://schema.org/")
xsd= Namespace("http://www.w3.org/2001/XMLSchema#")
owl= Namespace("http://www.w3.org/2002/07/owl#")

q1 = prepareQuery('''
SELECT DISTINCT ?chargingStation
WHERE 
{
  ?chargingStation a uiote:ChargingStation ;
}
  ''',
  initNs = { "ns": ns, "mv": mv, "uiote": uiote, "schema": schema, "xsd": xsd, "owl": owl}
)

for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_06a61e9dd36ed0944d8b626713954b58'),)
(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_0a6fee4b67d455129119a95d1f84b34f'),)
(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_1874b5f5a66f06509d9b6901ed7367d3'),)
(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_1a2fba2c73894c2417679bb6f86481d8'),)
(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_228925becaf736457758bb742e419701'),)
(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_243e02988fbbf9b61581b669dd41fd83'),)
(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_291709b8a506c6d21a47f5dd415c5087'),)
(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_2b6f7bd0f08f44d38c82a73c41c1931f'),)
(rdflib.term.URIRef('http://HO-G1.linkeddata.es/resource/Chargingstation/CS_2e17db05dd3cb942bde1

Repetimos la anterior consulta, pero pidiendo ahora además los nombres completos de los sujetos

In [13]:
q2 = prepareQuery('''
  SELECT ?Subject ?FullName WHERE { 
    ?Subject vcard:FN ?FullName.
  } 
  ''',
  initNs = { "vcard": VCARD}
)

for r in g.query(q2):
  print(r.Subject, r.FullName)

http://somewhere#JaneSmith Jane Smith
http://somewhere#JohnSmith John Smith
http://somewhere#SarahJones Sarah Jones
http://somewhere#MattJones Matt Jones


Obtenemos todos los recursos que contienen "Smith" como nombre de familia

In [10]:
from rdflib import XSD

q3 = prepareQuery('''
  SELECT ?Subject WHERE { 
    ?Subject vcard:Family ?Family.
  } 
  ''',
  initNs = { "vcard": VCARD}
)

for r in g.query(q3, initBindings = {'?Family' : Literal('Smith', datatype=XSD.string)}):
  print(r.Subject)

http://somewhere#JaneSmith
http://somewhere#JohnSmith


Obtenemos todos los elementos que contienen un email asociado

In [11]:
from rdflib import FOAF

q4 = prepareQuery('''
  SELECT ?Subject ?Email WHERE { 
    ?Subject foaf:email ?Email.
  } 
  ''',
  initNs = { "foaf": FOAF}
)

for r in g.query(q4):
  print(r.Subject,r.Email)

http://somewhere#JaneSmith jSmith@somewhere.com
http://somewhere#SarahJones sJones@somewhere.com


Consultamos todos los que conocen (FOAF:knows) a "Jane Smith" y obtenemos sus nombres de pila (VCARD:Given)

In [12]:
q5 = prepareQuery('''
  SELECT  ?Subject ?Given  WHERE {
    ?Subject foaf:knows ?JaneSmith.
	?JaneSmith vcard:FN ?JaneSmithFullName.
	?Subject vcard:Given ?Given.
  }  
  ''',
  initNs = { "foaf": FOAF, "vcard": VCARD, "xsd":XSD}
)

for r in g.query(q5, initBindings = {'?JaneSmithFullName' : Literal('Jane Smith', datatype=XSD.string)}):
  print(r.Subject, r.Given)

http://somewhere#JohnSmith John
http://somewhere#MattJones Matt
